In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import re
import joblib
import string


In [5]:
import pandas as pd

fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [9]:
fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
fake['class'] = 0
true['class'] = 1

In [11]:
data = pd.concat([fake, true], axis=0)

In [14]:
data.sample(10)

,text,class
9628,WASHINGTON (Reuters) - The White House said on...,1
2923,"At this point, it s fair to ask, What is Dona...",0
1295,NEW YORK/WASHINGTON (Reuters) - A Democratic U...,1
10421,Republican Senator Charles Grassley wants answ...,0
8082,WASHINGTON (Reuters) - Democratic presidential...,1
3371,CHICAGO (Reuters) - A U.S. judge could determi...,1
548,"WEST PALM BEACH, Fla. (Reuters) - U.S. Preside...",1
17340,ANKARA (Reuters) - Turkish security forces det...,1
7605,"Donald Trump claims to have a huge penis, but ...",0
19194,According to the Center for Immigration Studie...,0


In [13]:
data = data.drop(['title', 'subject', 'date'], axis=1)

In [15]:
data.reset_index(inplace=True)

In [16]:
data.drop(['index'], axis=1, inplace=True)

In [ ]:
data.sample(5)

,text,class
41209,GENEVA (Reuters) - Myanmar security forces hav...,1
24586,WASHINGTON (Reuters) - U.S. President Donald T...,1
39224,LUSAKA (Reuters) - Zambian President Edgar Lun...,1
12575,The days of the leftist media controlling the ...,0
22152,Lili Bayer and Larry Cohler-Esses HaaretzSebas...,0


In [18]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [21]:
import re
import string

data['text'] = data['text'].apply(clean_text)

In [22]:
data.sample(5)

,text,class
2493,after claiming that he s the least racist per...,0
38068,tokyo reuters japan s health ministry is l...,1
43428,madrid reuters the spanish government on f...,1
39960,berlin reuters the three parties exploring...,1
3835,trump fans may have won the election but they...,0


In [24]:
from sklearn.model_selection import train_test_split

x=data['text']
y=data['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)